# Predictive Model Part I: Preparing the data

For the first part of our predictive models, we use any possible feature without worrying about the interpretability, but focusing on having accurate guesses.

In this notebook, we prepare the data by applying dimensionality reduction techniques: PCA, spectral embedding, and t-SNE. Then, we normalize all the features. The output is another csv file saved as "data_dimred_normalized.csv".

Once the data is ready, considering the possible linear dependence of the features and to avoid overfitting, we will apply ElasticNet regression, KNN, and Neural Network models. In order to decide on model parameters and compare different models' accuracies, we use cross validation techniques like k-fold cross validation, LOOCV, and stratification regarding the small number of observations.

In [1]:
#Importing necessary packages

import os
import numpy as np
import sklearn
import pandas as pd
import random

#For plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import TSNE


from scipy import interpolate
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

path = '/Users/nesli/Desktop/SPRING2022/DSCC465/Project/'
os.chdir(path)

random.seed(265) #seed

## First, read the data [1]

In [2]:
data=pd.read_csv('us_county_sociohealth_data.csv')

In [3]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,Autauga,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,01003,Alabama,Baldwin,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,01005,Alabama,Barbour,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,01007,Alabama,Bibb,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,01009,Alabama,Blount,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,Wyoming,Uinta,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,Wyoming,Washakie,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,Wyoming,Weston,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,KSC,Missouri,Kansas City,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [4]:
list(data.columns)

['fips',
 'state',
 'county',
 'lat',
 'lon',
 'total_population',
 'area_sqmi',
 'population_density_per_sqmi',
 'num_deaths',
 'years_of_potential_life_lost_rate',
 'percent_fair_or_poor_health',
 'average_number_of_physically_unhealthy_days',
 'average_number_of_mentally_unhealthy_days',
 'percent_low_birthweight',
 'percent_smokers',
 'percent_adults_with_obesity',
 'food_environment_index',
 'percent_physically_inactive',
 'percent_with_access_to_exercise_opportunities',
 'percent_excessive_drinking',
 'num_alcohol_impaired_driving_deaths',
 'num_driving_deaths',
 'percent_driving_deaths_with_alcohol_involvement',
 'num_chlamydia_cases',
 'chlamydia_rate',
 'teen_birth_rate',
 'num_uninsured',
 'percent_uninsured',
 'num_primary_care_physicians',
 'primary_care_physicians_rate',
 'num_dentists',
 'dentist_rate',
 'num_mental_health_providers',
 'mental_health_provider_rate',
 'preventable_hospitalization_rate',
 'percent_with_annual_mammogram',
 'percent_vaccinated',
 'high_school

In [5]:
data['life_expectancy']

0       76.879477
1       78.450258
2       75.341935
3       73.571820
4       74.145826
          ...    
3139    79.245997
3140    79.451504
3141    80.549081
3142    78.173254
3143    83.232772
Name: life_expectancy, Length: 3144, dtype: float64

In [6]:
# Check for number of rows that have NaN [3]
data.isnull().any(axis=1).sum()

2717

In [7]:
# Check for number of rows that have NaN [3]
data['life_expectancy'].isnull().any(axis=0).sum()

1

In [8]:
# number of counties that miss life_expectancy data
len(np.where(data['life_expectancy'].isnull())[0]) 

73

In [9]:
len(np.where(data['life_expectancy'].isnull())[0])/len(data)

0.02321882951653944

In [10]:
# Print min and max of each row following from [4]
data.agg([min, max])

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
min,01001,Alabama,Abbeville,19.598724,-166.890423,76,2.050706,0.038490,32.0,2730.603992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,NYC,Wyoming,Ziebach,69.311848,-67.629217,10057155,145492.910697,72167.635613,84296.0,29138.378057,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [11]:
data['lon'].isnull().any(axis=0).sum()

0

In [12]:
# Maximum value in the entire dataframe [5]
max(data.max(axis=1))

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_40158/2223070932.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  max(data.max(axis=1))


10105518.0

In [13]:
# Minimum value in the entire dataframe [5]
min(data.min(axis=1))

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_40158/355634105.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  min(data.min(axis=1))


-166.890422684322

There are 2717 rows with null values in the data. Because we cannot get rid of rows, we could replace null values with either 0's or a very large number of our choice. We were inclined to pick a very large value to replace the null values with as we will already apply normalization before applying any model.  But the range of values change a lot from feature to feature so it would be hard to pick such a value.

Instead, we decide on replacing null values with a highly negative number. The only feature we see with negative values is "lon" and it has no null values. This is unlike what we have done in the explaratory part of our analysis and we choose to do so intentionally. Because here, we focus on building an accurate model and we choose not to make any assumptions about our data like a county would follow a similar distribution to the other counties in its state. We instead prefer to preserve the fact that a value is missing in the data by denoting it with highly different value than the other values in the data set. We choose the value "-1e6" regarding the maximum and minimum values in the entire data set. One caveat here is that the effect of this choice in each column would be different, however, this is not a problem thanks to the normalization we will soon be applying.

In [14]:
# Replace the null values with -1e6 [6]

data=data.fillna(-1e6)

In [15]:
# To check
data.isnull().any(axis=1).sum()

0

## Make columns with string values numerical

In [16]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,Autauga,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,01003,Alabama,Baldwin,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,01005,Alabama,Barbour,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,01007,Alabama,Bibb,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,01009,Alabama,Blount,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,Wyoming,Uinta,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,Wyoming,Washakie,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,Wyoming,Weston,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,KSC,Missouri,Kansas City,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [17]:
# Checking from the internet the fips codes for the missing counties [8]
# Missouri Kansas City 38000 [9]
data.at[3142,'fips']=38000

In [18]:
# Checking from the internet the fips codes for the missing counties [8]
# NYC 36061 [10]
data.at[3143,'fips']=36061

In [19]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,Autauga,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,01003,Alabama,Baldwin,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,01005,Alabama,Barbour,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,01007,Alabama,Bibb,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,01009,Alabama,Blount,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,Wyoming,Uinta,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,Wyoming,Washakie,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,Wyoming,Weston,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,38000,Missouri,Kansas City,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [20]:
# Replace state with a unique integer

len(np.unique(data['state']))

51

In [21]:
np.unique(data['state'])

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [22]:
# Replace county name with row index

for i in np.arange(len(data)):
    data.at[i,'county']=str(i)

In [23]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,0,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,01003,Alabama,1,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,01005,Alabama,2,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,01007,Alabama,3,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,01009,Alabama,4,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,Wyoming,3139,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,Wyoming,3140,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,Wyoming,3141,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,38000,Missouri,3142,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [24]:
# Convert the column data type [12]
data["county"] = pd.to_numeric(data["county"])

In [25]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,0,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,01003,Alabama,1,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,01005,Alabama,2,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,01007,Alabama,3,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,01009,Alabama,4,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,Wyoming,3139,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,Wyoming,3140,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,Wyoming,3141,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,38000,Missouri,3142,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [26]:
# From [11], returns the index of the value where the equality is satisfied
[j for j, x in enumerate(np.unique(data['state'])) if x == data.at[3143,'state']][0]

32

In [27]:
np.unique(data['state'])

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [28]:
state_list=list()

for i in np.arange(len(data)):
    state_list.append([j for j, x in enumerate(np.unique(data['state'])) if x == data.at[i,'state']][0])

In [29]:
state_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,


In [30]:
# Now, make the state column equal to this state_list

data['state']=state_list

In [31]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,0,0,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,01003,0,1,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,01005,0,2,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,01007,0,3,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,01009,0,4,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,50,3139,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,50,3140,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,50,3141,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,38000,25,3142,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [32]:
data.dtypes

fips                                                    object
state                                                    int64
county                                                   int64
lat                                                    float64
lon                                                    float64
                                                        ...   
percentile_rank_overcrowding                           float64
percentile_rank_no_vehicle                             float64
percentile_rank_institutionalized_in_group_quarters    float64
percentile_rank_housing_and_transportation             float64
percentile_rank_social_vulnerability                   float64
Length: 181, dtype: object

In [33]:
# Check if there is column of type string, but for pandas it is object [13]
[ col  for col, dt in data.dtypes.items() if dt == object]

['fips', 'presence_of_water_violation']

In [34]:
# Convert the column data type [12]
data["fips"] = pd.to_numeric(data["fips"])

In [35]:
data['presence_of_water_violation']

0       False
1       False
2       False
3       False
4       False
        ...  
3139     True
3140    False
3141    False
3142    False
3143    False
Name: presence_of_water_violation, Length: 3144, dtype: object

In [36]:
pd.to_numeric(data['presence_of_water_violation'])

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3139    1.0
3140    0.0
3141    0.0
3142    0.0
3143    0.0
Name: presence_of_water_violation, Length: 3144, dtype: float64

In [37]:
# Convert the column data type [12]
data['presence_of_water_violation'] = pd.to_numeric(data['presence_of_water_violation'])

In [38]:
data

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,1001,0,0,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,1003,0,1,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,1005,0,2,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,1007,0,3,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,1009,0,4,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,50,3139,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,50,3140,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,50,3141,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,38000,25,3142,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [39]:
# Check if there is column of type string, but for pandas it is object [13]
[ col  for col, dt in data.dtypes.items() if dt == object]

[]

Before applying normalization or dimensionality reduction, we should store the target variable of 'life_expectancy' in  a file and remove it from the data set.

In [40]:
y=data['life_expectancy']

In [41]:
X=data

In [42]:
del X['life_expectancy']

In [43]:
y.to_csv('y.csv')

In [44]:
X

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,1001,0,0,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,1003,0,1,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,1005,0,2,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,1007,0,3,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,1009,0,4,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56041,50,3139,41.287648,-110.547639,20893,2081.827988,10.035892,256.0,7330.683549,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,56043,50,3140,43.904970,-107.682819,8351,2238.550443,3.730539,110.0,6586.049959,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,56045,50,3141,43.840417,-104.567663,7175,2398.090296,2.991964,89.0,5388.706947,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,38000,25,3142,39.130000,-94.550000,1113072,2118.989867,525.284248,14267.0,7770.929831,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


## Normalize the data

Before applying any dimensionality technique, we normalize the data. For normalization, we choose to normalize with min-max scaling instead of assuming that the data in a column follows a normal distribution.

In [45]:
# Following from [14]

data_min_max_scaled = X.copy()

for column in data_min_max_scaled.columns:
    data_min_max_scaled[column] = (data_min_max_scaled[column] - data_min_max_scaled[column].min()) / (data_min_max_scaled[column].max() - data_min_max_scaled[column].min())    

In [46]:
data_min_max_scaled

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,0.000000,0.00,0.000000,0.260217,0.808450,0.005466,0.004072,0.001283,0.922987,0.979585,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,0.000036,0.00,0.000318,0.223860,0.797571,0.019830,0.010913,0.001738,0.924994,0.978833,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,0.000073,0.00,0.000636,0.246834,0.821038,0.002639,0.006068,0.000416,0.922693,0.981650,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,0.000109,0.00,0.000955,0.269545,0.803576,0.002237,0.004265,0.000502,0.922692,0.983325,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,0.000145,0.00,0.001273,0.289303,0.809209,0.005730,0.004418,0.001240,0.923258,0.982701,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.999927,1.00,0.998727,0.436282,0.567621,0.002070,0.014295,0.000139,0.922493,0.978810,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,0.999964,1.00,0.999045,0.488930,0.596483,0.000823,0.015372,0.000051,0.922359,0.978086,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,1.000000,1.00,0.999364,0.487632,0.627866,0.000706,0.016469,0.000041,0.922339,0.976923,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,0.672171,0.50,0.999682,0.392880,0.728788,0.110668,0.014550,0.007278,0.935415,0.979238,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [47]:
X=data_min_max_scaled

In [48]:
data_scaled=data_min_max_scaled

In [49]:
del data_min_max_scaled

In [50]:
X

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,0.000000,0.00,0.000000,0.260217,0.808450,0.005466,0.004072,0.001283,0.922987,0.979585,...,0.633900,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300
1,0.000036,0.00,0.000318,0.223860,0.797571,0.019830,0.010913,0.001738,0.924994,0.978833,...,0.525300,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700
2,0.000073,0.00,0.000636,0.246834,0.821038,0.002639,0.006068,0.000416,0.922693,0.981650,...,0.904200,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700
3,0.000109,0.00,0.000955,0.269545,0.803576,0.002237,0.004265,0.000502,0.922692,0.983325,...,0.645000,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700
4,0.000145,0.00,0.001273,0.289303,0.809209,0.005730,0.004418,0.001240,0.923258,0.982701,...,0.423800,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.999927,1.00,0.998727,0.436282,0.567621,0.002070,0.014295,0.000139,0.922493,0.978810,...,0.421200,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200
3140,0.999964,1.00,0.999045,0.488930,0.596483,0.000823,0.015372,0.000051,0.922359,0.978086,...,0.536100,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800
3141,1.000000,1.00,0.999364,0.487632,0.627866,0.000706,0.016469,0.000041,0.922339,0.976923,...,0.288800,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600
3142,0.672171,0.50,0.999682,0.392880,0.728788,0.110668,0.014550,0.007278,0.935415,0.979238,...,0.675575,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899


In [51]:
# Add life_expectancy column back for plotting purposes
X['life_expectancy']=y

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_40158/464551723.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['life_expectancy']=y


In [52]:
X

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability,life_expectancy
0,0.000000,0.00,0.000000,0.260217,0.808450,0.005466,0.004072,0.001283,0.922987,0.979585,...,0.535500,0.597600,0.679100,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300,76.879477
1,0.000036,0.00,0.000318,0.223860,0.797571,0.019830,0.010913,0.001738,0.924994,0.978833,...,0.528200,0.529400,0.973300,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700,78.450258
2,0.000073,0.00,0.000636,0.246834,0.821038,0.002639,0.006068,0.000416,0.922693,0.981650,...,0.697900,0.855800,0.281400,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700,75.341935
3,0.000109,0.00,0.000955,0.269545,0.803576,0.002237,0.004265,0.000502,0.922692,0.983325,...,0.355300,0.501800,0.407200,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700,73.571820
4,0.000145,0.00,0.001273,0.289303,0.809209,0.005730,0.004418,0.001240,0.923258,0.982701,...,0.748200,0.599200,0.134400,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600,74.145826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.999927,1.00,0.998727,0.436282,0.567621,0.002070,0.014295,0.000139,0.922493,0.978810,...,0.662500,0.547600,0.770500,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200,79.245997
3140,0.999964,1.00,0.999045,0.488930,0.596483,0.000823,0.015372,0.000051,0.922359,0.978086,...,0.464200,0.502100,0.346400,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800,79.451504
3141,1.000000,1.00,0.999364,0.487632,0.627866,0.000706,0.016469,0.000041,0.922339,0.976923,...,0.620200,0.442500,0.552100,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600,80.549081
3142,0.672171,0.50,0.999682,0.392880,0.728788,0.110668,0.014550,0.007278,0.935415,0.979238,...,0.669271,0.705512,0.879288,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899,78.173254


## Dimensionality Reduction

I had written a similar code for my Kaggle project in [2] and here we apply similar methods so we follow from there.

### PCA

In [53]:
#Following from [7] to use sklearn's PCA 

sklearn_pca = PCA(n_components=3)
pcs = sklearn_pca.fit_transform(data_scaled)

In [54]:
pcs[:,0]

array([-23220.04050546, -23221.6112868 , -23218.50296316, ...,
       -23223.71009956, -23221.33428262, -23226.39380089])

In [55]:
X['pca_pc1']=pcs[:,0]
X['pca_pc2']=pcs[:,1]
X['pca_pc3']=pcs[:,2]

In [56]:
X

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability,life_expectancy,pca_pc1,pca_pc2,pca_pc3
0,0.000000,0.00,0.000000,0.260217,0.808450,0.005466,0.004072,0.001283,0.922987,0.979585,...,0.726800,0.247700,0.329800,0.125100,0.288100,0.377300,76.879477,-23220.040505,-1.227569,-0.246848
1,0.000036,0.00,0.000318,0.223860,0.797571,0.019830,0.010913,0.001738,0.924994,0.978833,...,0.538700,0.263900,0.087200,0.343800,0.332400,0.275700,78.450258,-23221.611287,-1.396633,-0.888378
2,0.000073,0.00,0.000636,0.246834,0.821038,0.002639,0.006068,0.000416,0.922693,0.981650,...,0.937000,0.443800,0.881600,0.942700,0.931200,0.984700,75.341935,-23218.502963,-0.752975,2.034054
3,0.000109,0.00,0.000955,0.269545,0.803576,0.002237,0.004265,0.000502,0.922692,0.983325,...,0.924900,0.024800,0.564500,0.915600,0.666300,0.573700,73.571820,-23216.732849,-1.176112,0.475322
4,0.000145,0.00,0.001273,0.289303,0.809209,0.005730,0.004418,0.001240,0.923258,0.982701,...,0.846500,0.505600,0.190700,0.151500,0.182700,0.498600,74.145826,-23217.306855,-1.406594,0.105359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.999927,1.00,0.998727,0.436282,0.567621,0.002070,0.014295,0.000139,0.922493,0.978810,...,0.875800,0.829000,0.126100,0.215900,0.656200,0.535200,79.245997,-23222.407022,0.208494,-0.024748
3140,0.999964,1.00,0.999045,0.488930,0.596483,0.000823,0.015372,0.000051,0.922359,0.978086,...,0.511600,0.583900,0.580100,0.537400,0.514200,0.592800,79.451504,-23222.612525,1.798838,0.229763
3141,1.000000,1.00,0.999364,0.487632,0.627866,0.000706,0.016469,0.000041,0.922339,0.976923,...,0.861800,0.177000,0.381400,0.807700,0.633900,0.271600,80.549081,-23223.710100,2.407547,-0.657690
3142,0.672171,0.50,0.999682,0.392880,0.728788,0.110668,0.014550,0.007278,0.935415,0.979238,...,0.055045,0.358059,0.613377,0.329812,0.384675,0.392899,78.173254,-23221.334283,-1.597692,-0.753245


In [57]:
# visualization_code.py
# Directly copy-pasted from the provided file for earlier homework [15], and also [16]

mpl.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 10

df = X
#pal = sns.color_palette("Paired")[:len(set(X['life_expectancy']))]
pal=sns.color_palette("Spectral", as_cmap=True)
p1 = sns.scatterplot(x="pca_pc1", y='pca_pc2', hue='life_expectancy', palette = pal, data=df, s=250, alpha=0.7, legend=False)

#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
     p1.text(df.pca_pc1[line], df.pca_pc2[line], df.life_expectancy[line], horizontalalignment='left', size='medium', color='black', weight='semibold')

plt.suptitle('Two-Dimensional Map (PCA)', fontsize=36)
plt.xlabel('Dimension 1', fontsize=24)
plt.ylabel('Dimension 2', fontsize=24)

#for i in df.life_expectancy.unique():
    # get the convex hull
#    points = df[df.life_expectancy == i][['pca_pc1', 'pca_pc2']].values
#    hull = ConvexHull(points)
#    x_hull = np.append(points[hull.vertices,0],
#                       points[hull.vertices,0][0])
#    y_hull = np.append(points[hull.vertices,1],
#                       points[hull.vertices,1][0])
#    
    # interpolate
#    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
#    dist_along = np.concatenate(([0], dist.cumsum()))
#    spline, u = interpolate.splprep([x_hull, y_hull], 
#                                    u=dist_along, s=0)
#    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
#    interp_x, interp_y = interpolate.splev(interp_d, spline)
#    # plot shape
#    plt.fill(interp_x, interp_y, '--', c=pal[i], alpha=0.2)
      
    
plt.grid()
plt.show()

PCA returns 2 clusters when we plot the whole data in 2 dimensions. 'life_expectancy' column is not normalized and missing values are set to '-1e6' in the whole data set. However, because 'life_expectancy' column is not included in the PCA calculation, we could still expect normalization to take care of the difference in values of the data set. Yet this PCA plot shows that missing 'life_expectancy' counties are much different than the rest. This might be because the rows that miss 'life_expectancy' probably also miss some other column information. This shows that missing data was actually important and although we can instead replace it with something that makes sense, we actually miss some significant part of the data. 

This also shows that there are some number of counties with missing values for the 'life_expectancy' so we take a step back to replace values with the mean for the state just like was done in the explanatory part. But we include that part in the code because that clearly shows that we should replace missing values with something close to rest of the data. In fact, there are 73 counties with missing 'life_expectancy' data which corresponds to $2\%$ of the data.

For the explanatory part of the analysis, the missing data was replaced with mean, so we follow from there.  But regarding the predictions that will be performed and the fact that there can be outliers in the data, we think that replacing missing values with median could be more robust to outliers and that remains as an option to explore later if our models do not perform well enough.

We end this notebook here and instead, perform the same analysis except for changing the approach to missing data to prepare the data in another notebook.

## References:

[1] https://www.kaggle.com/datasets/johnjdavisiv/us-counties-covid19-weather-sociohealth-data?select=us_county_sociohealth_data.csv

[2] Kaggle project's prepare_data.ipynb by Erez and Kibris

[3] https://stackoverflow.com/questions/28199524/best-way-to-count-the-number-of-rows-with-missing-values-in-a-pandas-dataframe

[4] https://stackoverflow.com/questions/29276301/max-and-min-value-for-each-colum-of-one-dataframe

[5] https://stackoverflow.com/questions/24571005/return-max-value-from-pandas-dataframe-as-a-whole-not-based-on-column-or-rows

[6] https://datatofish.com/replace-nan-values-with-zeros/

[7] Galarnyk, M. (2021, November 17). PCA using Python (scikit-learn). Medium. Retrieved April 1, 2022, from https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60 

[8] https://www.askpython.com/python-modules/pandas/update-the-value-of-a-row-dataframe

[9] https://health.mo.gov/data/geocodes/geocodelist2.php

[10] https://guides.newman.baruch.cuny.edu/nyc_data#:~:text=Manhattan%20is%20New%20York%20County%20(ANSI%20%2F%20FIPS%2036061)

[11] https://stackoverflow.com/questions/6294179/how-to-find-all-occurrences-of-an-element-in-a-list

[12] https://stackoverflow.com/questions/15891038/change-column-type-in-pandas#:~:text=1.-,to_numeric(),floating%2Dpoint%20numbers%20as%20appropriate.

[13] https://stackoverflow.com/questions/43049545/python-check-if-dataframe-column-contain-string-type

[14] https://www.geeksforgeeks.org/data-normalization-with-pandas/

[15] visualization_code.py for Homework 5